In [1]:
from pyspark import SparkContext
from pyspark import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.getOrCreate()

from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import isnan, when, count, col

In [2]:
sdfHolidayEvents = sqlContext.read.csv("/gl-capstone-data/Team6-C-Sep/Data/holidays_events_2013_2015.csv",header = True,inferSchema = True)

In [4]:
sdfHolidayEvents.count()

170

In [5]:
pdfHolidayEvents = sdfHolidayEvents.toPandas()

In [6]:
sqlContext.registerDataFrameAsTable(sdfHolidayEvents, "sdfHolidayEventsTbl")

In [7]:
sdfHolidayEvents1 = sqlContext.sql("select date, count(*) from sdfHolidayEventsTbl group by date order by 2 desc")
sdfHolidayEvents1.show()

+-------------------+--------+
|               date|count(1)|
+-------------------+--------+
|2014-06-25 00:00:00|       4|
|2013-06-25 00:00:00|       3|
|2015-06-25 00:00:00|       3|
|2013-07-03 00:00:00|       2|
|2014-12-22 00:00:00|       2|
|2015-07-03 00:00:00|       2|
|2013-12-22 00:00:00|       2|
|2014-12-26 00:00:00|       2|
|2015-12-22 00:00:00|       2|
|2014-07-03 00:00:00|       2|
|2013-05-12 00:00:00|       2|
|2014-08-15 00:00:00|       1|
|2013-04-01 00:00:00|       1|
|2013-10-11 00:00:00|       1|
|2013-09-28 00:00:00|       1|
|2013-08-05 00:00:00|       1|
|2015-03-02 00:00:00|       1|
|2013-08-24 00:00:00|       1|
|2015-11-06 00:00:00|       1|
|2014-07-09 00:00:00|       1|
+-------------------+--------+
only showing top 20 rows



In [9]:
sdfHolidayEvents1.count()

155

In [10]:
sdfHolidayEvents2 = sqlContext.sql("select * from sdfHolidayEventsTbl where date not in ('2014-06-25 00:00:00', '2013-06-25 00:00:00','2015-06-25 00:00:00','2013-07-03 00:00:00','2014-12-22 00:00:00','2015-07-03 00:00:00','2013-12-22 00:00:00','2014-12-26 00:00:00','2015-12-22 00:00:00','2014-07-03 00:00:00','2013-05-12 00:00:00') order by date")
sdfHolidayEvents2.show()

+-------------------+----------+--------+-----------+--------------------+-----------+
|               date|      type|  locale|locale_name|         description|transferred|
+-------------------+----------+--------+-----------+--------------------+-----------+
|2013-01-01 00:00:00|   Holiday|National|    Ecuador|  Primer dia del ano|      false|
|2013-01-05 00:00:00|  Work Day|National|    Ecuador|Recupero puente N...|      false|
|2013-01-12 00:00:00|  Work Day|National|    Ecuador|Recupero puente p...|      false|
|2013-02-11 00:00:00|   Holiday|National|    Ecuador|            Carnaval|      false|
|2013-02-12 00:00:00|   Holiday|National|    Ecuador|            Carnaval|      false|
|2013-03-02 00:00:00|   Holiday|   Local|      Manta|  Fundacion de Manta|      false|
|2013-04-01 00:00:00|   Holiday|Regional|   Cotopaxi|Provincializacion...|      false|
|2013-04-12 00:00:00|   Holiday|   Local|     Cuenca| Fundacion de Cuenca|      false|
|2013-04-14 00:00:00|   Holiday|   Local|  

In [11]:
pdfHolidayEventsWithNoDuplicates = sdfHolidayEvents2.toPandas()

In [12]:
pdfHolidayEventsWithNoDuplicates.shape[0]

144

In [13]:
pdfHolidayEventsWithNoDuplicates.head(3)

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False


In [14]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2014-06-25']

,date,type,locale,locale_name,description,transferred
69,2014-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
70,2014-06-25,Holiday,Local,Machala,Fundacion de Machala,False
71,2014-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
72,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False


In [15]:
rec = pdfHolidayEvents[pdfHolidayEvents.index == 72]
rec

,date,type,locale,locale_name,description,transferred
72,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False


In [16]:
pdfHolidayEventsWithNoDuplicates.loc[pdfHolidayEventsWithNoDuplicates.shape[0]]  = [rec.iloc[0]['date'], rec.iloc[0]['type'], rec.iloc[0]['locale'], rec.iloc[0]['locale_name'], rec.iloc[0]['description'], rec.iloc[0]['transferred']]

In [17]:
pdfHolidayEventsWithNoDuplicates.tail(3)

,date,type,locale,locale_name,description,transferred
142,2015-12-26,Additional,National,Ecuador,Navidad+1,False
143,2015-12-31,Additional,National,Ecuador,Primer dia del ano-1,False
144,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False


In [18]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2013-06-25']

,date,type,locale,locale_name,description,transferred
17,2013-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
18,2013-06-25,Holiday,Local,Machala,Fundacion de Machala,False
19,2013-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False


In [19]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2015-06-25']

,date,type,locale,locale_name,description,transferred
135,2015-06-25,Holiday,Local,Machala,Fundacion de Machala,False
136,2015-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
137,2015-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False


In [20]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2013-07-03']

,date,type,locale,locale_name,description,transferred
20,2013-07-03,Holiday,Local,El Carmen,Cantonizacion de El Carmen,False
21,2013-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False


In [21]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2014-12-22']

,date,type,locale,locale_name,description,transferred
110,2014-12-22,Holiday,Local,Salinas,Cantonizacion de Salinas,False
111,2014-12-22,Additional,National,Ecuador,Navidad-3,False


In [22]:
rec = pdfHolidayEvents[pdfHolidayEvents.index == 111]
rec

,date,type,locale,locale_name,description,transferred
111,2014-12-22,Additional,National,Ecuador,Navidad-3,False


In [23]:
pdfHolidayEventsWithNoDuplicates.loc[pdfHolidayEventsWithNoDuplicates.shape[0]]  = [rec.iloc[0]['date'], rec.iloc[0]['type'], rec.iloc[0]['locale'], rec.iloc[0]['locale_name'], rec.iloc[0]['description'], rec.iloc[0]['transferred']]

In [24]:
pdfHolidayEventsWithNoDuplicates.tail(3)

,date,type,locale,locale_name,description,transferred
143,2015-12-31,Additional,National,Ecuador,Primer dia del ano-1,False
144,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False
145,2014-12-22,Additional,National,Ecuador,Navidad-3,False


In [25]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2015-07-03']

,date,type,locale,locale_name,description,transferred
138,2015-07-03,Holiday,Local,El Carmen,Cantonizacion de El Carmen,False
139,2015-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False


In [26]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2013-12-22']

,date,type,locale,locale_name,description,transferred
44,2013-12-22,Additional,National,Ecuador,Navidad-3,False
45,2013-12-22,Holiday,Local,Salinas,Cantonizacion de Salinas,False


In [27]:
rec = pdfHolidayEvents[pdfHolidayEvents.index == 44]
rec

,date,type,locale,locale_name,description,transferred
44,2013-12-22,Additional,National,Ecuador,Navidad-3,False


In [28]:
pdfHolidayEventsWithNoDuplicates.loc[pdfHolidayEventsWithNoDuplicates.shape[0]]  = [rec.iloc[0]['date'], rec.iloc[0]['type'], rec.iloc[0]['locale'], rec.iloc[0]['locale_name'], rec.iloc[0]['description'], rec.iloc[0]['transferred']]

In [29]:
pdfHolidayEventsWithNoDuplicates.tail(3)

,date,type,locale,locale_name,description,transferred
144,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False
145,2014-12-22,Additional,National,Ecuador,Navidad-3,False
146,2013-12-22,Additional,National,Ecuador,Navidad-3,False


In [30]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2014-12-26']

,date,type,locale,locale_name,description,transferred
115,2014-12-26,Bridge,National,Ecuador,Puente Navidad,False
116,2014-12-26,Additional,National,Ecuador,Navidad+1,False


In [31]:
rec = pdfHolidayEvents[pdfHolidayEvents.index == 116]
rec

,date,type,locale,locale_name,description,transferred
116,2014-12-26,Additional,National,Ecuador,Navidad+1,False


In [32]:
pdfHolidayEventsWithNoDuplicates.loc[pdfHolidayEventsWithNoDuplicates.shape[0]]  = [rec.iloc[0]['date'], rec.iloc[0]['type'], rec.iloc[0]['locale'], rec.iloc[0]['locale_name'], rec.iloc[0]['description'], rec.iloc[0]['transferred']]

In [33]:
pdfHolidayEventsWithNoDuplicates.tail(3)

,date,type,locale,locale_name,description,transferred
145,2014-12-22,Additional,National,Ecuador,Navidad-3,False
146,2013-12-22,Additional,National,Ecuador,Navidad-3,False
147,2014-12-26,Additional,National,Ecuador,Navidad+1,False


In [34]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2015-12-22']

,date,type,locale,locale_name,description,transferred
163,2015-12-22,Additional,National,Ecuador,Navidad-3,False
164,2015-12-22,Holiday,Local,Salinas,Cantonizacion de Salinas,False


In [35]:
rec = pdfHolidayEvents[pdfHolidayEvents.index == 163]
rec

,date,type,locale,locale_name,description,transferred
163,2015-12-22,Additional,National,Ecuador,Navidad-3,False


In [36]:
pdfHolidayEventsWithNoDuplicates.loc[pdfHolidayEventsWithNoDuplicates.shape[0]]  = [rec.iloc[0]['date'], rec.iloc[0]['type'], rec.iloc[0]['locale'], rec.iloc[0]['locale_name'], rec.iloc[0]['description'], rec.iloc[0]['transferred']]

In [37]:
pdfHolidayEventsWithNoDuplicates.tail(3)

,date,type,locale,locale_name,description,transferred
146,2013-12-22,Additional,National,Ecuador,Navidad-3,False
147,2014-12-26,Additional,National,Ecuador,Navidad+1,False
148,2015-12-22,Additional,National,Ecuador,Navidad-3,False


In [38]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2014-07-03']

,date,type,locale,locale_name,description,transferred
77,2014-07-03,Holiday,Local,El Carmen,Cantonizacion de El Carmen,False
78,2014-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False


In [39]:
pdfHolidayEvents[pdfHolidayEvents['date'] == '2013-05-12']

,date,type,locale,locale_name,description,transferred
13,2013-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
14,2013-05-12,Event,National,Ecuador,Dia de la Madre,False


In [40]:
rec = pdfHolidayEvents[pdfHolidayEvents.index == 14]
rec

,date,type,locale,locale_name,description,transferred
14,2013-05-12,Event,National,Ecuador,Dia de la Madre,False


In [41]:
pdfHolidayEventsWithNoDuplicates.loc[pdfHolidayEventsWithNoDuplicates.shape[0]]  = [rec.iloc[0]['date'], rec.iloc[0]['type'], rec.iloc[0]['locale'], rec.iloc[0]['locale_name'], rec.iloc[0]['description'], rec.iloc[0]['transferred']]

In [42]:
pdfHolidayEventsWithNoDuplicates.tail(3)

,date,type,locale,locale_name,description,transferred
147,2014-12-26,Additional,National,Ecuador,Navidad+1,False
148,2015-12-22,Additional,National,Ecuador,Navidad-3,False
149,2013-05-12,Event,National,Ecuador,Dia de la Madre,False


In [43]:
pdfHolidayEventsWithNoDuplicates.head(3)

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False


In [44]:
pdfHolidayEventsWithNoDuplicates.tail(12)

,date,type,locale,locale_name,description,transferred
138,2015-12-21,Additional,National,Ecuador,Navidad-4,False
139,2015-12-23,Additional,National,Ecuador,Navidad-2,False
140,2015-12-24,Additional,National,Ecuador,Navidad-1,False
141,2015-12-25,Holiday,National,Ecuador,Navidad,False
142,2015-12-26,Additional,National,Ecuador,Navidad+1,False
143,2015-12-31,Additional,National,Ecuador,Primer dia del ano-1,False
144,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False
145,2014-12-22,Additional,National,Ecuador,Navidad-3,False
146,2013-12-22,Additional,National,Ecuador,Navidad-3,False
147,2014-12-26,Additional,National,Ecuador,Navidad+1,False


In [45]:
pdfHolidayEventsWithNoDuplicates.dtypes

date           datetime64[ns]
type                   object
locale                 object
locale_name            object
description            object
transferred              bool
dtype: object

In [46]:
pdfHolidayEventsWithNoDuplicates['transferred'] = pdfHolidayEventsWithNoDuplicates['transferred'].astype(str)

In [47]:
pdfHolidayEventsWithNoDuplicates.head(3)

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False


In [48]:
pdfHolidayEventsWithNoDuplicates['date'] = pdfHolidayEventsWithNoDuplicates['date'].astype(str)
# Added to have the date saved in "2013-01-01" format instead of integer

In [49]:
pdfHolidayEventsWithNoDuplicates.head(3)

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False


In [50]:
pdfHolidayEventsWithNoDuplicates.tail(3)

,date,type,locale,locale_name,description,transferred
147,2014-12-26,Additional,National,Ecuador,Navidad+1,False
148,2015-12-22,Additional,National,Ecuador,Navidad-3,False
149,2013-05-12,Event,National,Ecuador,Dia de la Madre,False


In [51]:
pdfHolidayEventsWithNoDuplicates.shape

(150, 6)

In [52]:
pdfHolidayEventsWithNoDuplicates.dtypes

date           object
type           object
locale         object
locale_name    object
description    object
transferred    object
dtype: object

In [53]:
sdfResult = sqlContext.createDataFrame(pdfHolidayEventsWithNoDuplicates)

In [54]:
type(sdfResult)

pyspark.sql.dataframe.DataFrame

In [55]:
sdfResult.show(5)

+----------+--------+--------+-----------+--------------------+-----------+
|      date|    type|  locale|locale_name|         description|transferred|
+----------+--------+--------+-----------+--------------------+-----------+
|2013-01-01| Holiday|National|    Ecuador|  Primer dia del ano|      False|
|2013-01-05|Work Day|National|    Ecuador|Recupero puente N...|      False|
|2013-01-12|Work Day|National|    Ecuador|Recupero puente p...|      False|
|2013-02-11| Holiday|National|    Ecuador|            Carnaval|      False|
|2013-02-12| Holiday|National|    Ecuador|            Carnaval|      False|
+----------+--------+--------+-----------+--------------------+-----------+
only showing top 5 rows



In [56]:
sdfResult.count()

150

In [57]:
sdfResult.printSchema()

root
 |-- date: string (nullable = true)
 |-- type: string (nullable = true)
 |-- locale: string (nullable = true)
 |-- locale_name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- transferred: string (nullable = true)



In [58]:
sdfResult.write.csv("/gl-capstone-data/Team6-C-Sep/Data/holidays_events_2013_2015_noduplicates.csv", header=True)

In [59]:
sdfResultReload = sqlContext.read.csv("/gl-capstone-data/Team6-C-Sep/Data/holidays_events_2013_2015_noduplicates.csv",header = True,inferSchema = True)

In [61]:
sdfResultReload.count()

150

In [62]:
sdfResultReload.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- type: string (nullable = true)
 |-- locale: string (nullable = true)
 |-- locale_name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- transferred: boolean (nullable = true)



In [63]:
print(sdfResultReload.show(5))

+-------------------+--------+--------+-----------+--------------------+-----------+
|               date|    type|  locale|locale_name|         description|transferred|
+-------------------+--------+--------+-----------+--------------------+-----------+
|2013-01-01 00:00:00| Holiday|National|    Ecuador|  Primer dia del ano|      false|
|2013-01-05 00:00:00|Work Day|National|    Ecuador|Recupero puente N...|      false|
|2013-01-12 00:00:00|Work Day|National|    Ecuador|Recupero puente p...|      false|
|2013-02-11 00:00:00| Holiday|National|    Ecuador|            Carnaval|      false|
|2013-02-12 00:00:00| Holiday|National|    Ecuador|            Carnaval|      false|
+-------------------+--------+--------+-----------+--------------------+-----------+
only showing top 5 rows

None
